# Riemann problem

## libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from matplotlib import animation
from IPython.display import HTML
from scipy import special

## initialization

In [4]:
X_max = 15
T_max = 100
X_min = T_min = 0

NUMBER_X = 50
NUMBER_T = 100

In [6]:
X_RANGE = np.linspace(X_min, X_max, NUMBER_X)
T_RANGE = np.linspace(T_min, T_max, NUMBER_T)

X_STEP = X_RANGE[1] - X_RANGE[0]
T_STEP = T_RANGE[1] - T_RANGE[0]

In [15]:
VELOCITY_left = 300
DENSITY_left = 10
PRESSURE_left = 200
ENERGY_LEFT = 100

In [16]:
def set_conditions(f, BOUNDARY_CONDITION):
    f[:, 0] = 0
    f[0, :] = BOUNDARY_CONDITION
    f[-1, :] = 0
    
    return f

In [20]:
X, T = np.meshgrid(X_RANGE, T_RANGE, indexing='ij')

# Boundary / initial conditions
velocity, density, pressure, energy = [np.zeros_like(X)] * 4

velocity = set_conditions(velocity, VELOCITY_left)
density = set_conditions(density, DENSITY_left)
pressure = set_conditions(pressure, PRESSURE_left)
energy = set_conditions(energy, ENERGY_LEFT)

In [29]:
def euler_step(f_a, f_b, f_c):
    """f_a is f[i-1], f_b is f[i], f_c is f[i+1]"""
    
    f_plus = (f_b + f_c) / 2
    f_minus = (f_a + f_b) / 2
    
    return f_plus, f_minus

def lagrange_step(r_a, r_b, r_c, u_plus, u_minus):
    """r_a is density[i-1], r_b is density[i], r_c is density[i+1]""" 
    
    if u_plus >= 0:
        M_plus = r_b * u_plus * T_STEP
    else:
        M_plus = r_c * u_plus * T_STEP
        
    if u_minus >= 0:
        M_minus = r_a * u_minus * T_STEP
    else:
        M_plus = r_b * u_minus * T_STEP
        
    return abs(M_plus), abs(M_minus)

def final_step(Y_plus, Y_minus, r_i, u, e, u_plus, u_minus):
    
    if u_plus >= 0: 
        D_plus = 0
    else: 
        D_plus = 1
        
    if u_minus >= 0: 
        D_minus = 1
    else: 
        D_plus = 0
        
    r_next = r_i + 2 * ((D_minus - 0.5) * Y_minus + (D_plus - 0.5) * Y_plus) / X_STEP

## main procedure